In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '../terrace')

%load_ext autoreload
%autoreload 2

In [77]:
import pickle
import torch
import wandb
import pandas as pd
from glob import glob
from git_timewarp import GitTimeWarp
from tqdm import tqdm
import random

from datasets.graphs.mol_graph import MolGraph, mol_graph_from_sdf
from datasets.graphs.prot_graph import ProtGraph, prot_graph_from_pdb
from datasets.data_types import ActivityData, IsActiveData
from terrace.batch import make_batch
from routines.ai_routine import AIRoutine
from common.cfg_utils import get_config, get_run_config

from screen import *

In [50]:
cfg = get_config("../configs", "short_thicc_op_gnn")
run_ids = ["1nhqz8vw"] #, "1socj7qg" ]
name2model = get_name_to_model(cfg, run_ids)

Loading models...


In [52]:
for screen_csv in glob(cfg.platform.bigbind_dir + "/val_screens/*.csv"):
    break

In [53]:
screen_df = pd.read_csv(screen_csv)
batches = get_batches(cfg, screen_df)
preds = []
with torch.no_grad():
    for batch in tqdm(batches):
        preds.append(model(batch))

Getting batches for CAH2_HUMAN_2_260_0


100%|██████████| 996/996 [01:06<00:00, 14.97it/s]


In [67]:
cache_folder = cfg.platform.cache_dir + "/screen_preds/" + screen_df.pocket[0] + "/"
cache_file = cache_folder + name + ".pkl"
os.makedirs(cache_folder, exist_ok=True)

In [90]:
results = screen_all(cfg, run_ids)

Loading models...
Screening CAH2_HUMAN_2_260_0
Getting batches for CAH2_HUMAN_2_260_0
Predicting with short_thicc


In [97]:
pd.DataFrame({"aaaa": {'b': 4}, "bbbb": {"b": 4}})

,aaaa,bbbb
b,4,4


In [142]:
def results_to_df(results):
    rows = []
    for pocket, poc_res in results.items():
        row = { "pocket": pocket }
        for model_name, metrics in poc_res.items():
            for met_name, metric in metrics.items():
                row[f"{model_name}_{met_name}"] = metric
        rows.append(row)
    return pd.DataFrame(rows)
results_to_df(results)

,pocket,short_thicc_efi,short_thicc_nefi,rand_efi,rand_nefi
0,CAH2_HUMAN_2_260_0,0.1,0.1,0.0,0.0
1,AAAAAA,0.1,0.1,0.0,0.0


In [141]:
results["AAAAAA"] = results["CAH2_HUMAN_2_260_0"]

In [125]:
results

{'CAH2_HUMAN_2_260_0': {'short_thicc': {'efi': 0.1, 'nefi': 0.1},
  'rand': {'efi': 0.0, 'nefi': 0.0}},
 'AAAAAA': {'short_thicc': {'efi': 0.1, 'nefi': 0.1},
  'rand': {'efi': 0.0, 'nefi': 0.0}}}

In [145]:
def save_results(results):
    os.makedirs("outputs", exist_ok=True)
    df = results_to_df(results)
    out_file = "outputs/val_screen_results.csv"
    print(out_file)
    df.to_csv(out_file, index=False)
save_results(results)

outputs/val_screen_results.csv
